In [1]:
import pandas as pd
import obtenerDatos as oD
import conexionSGBD as cS
import util as ut
import os

 Obtenemos todas las estaciones climáticas que tiene la CAR. Para ello nos bajamos el catálogo de estaciones a través del siguiente enlace:
 https://ias1.larioja.org/apiSiar/servicios/v2/estaciones
 El formato que viene es json, adaptamos este json a nuestra tabla Estaciones. 

In [ ]:
## Definimos el directorio de trabajo
#os.chdir('D:\\Plagas')
dfEstaciones=pd.read_json('estaciones.json')
## Seleccionamos solo las columnas que nos interesan.
dfEstaciones=dfEstaciones[['codigo_estacion','municipio','nombre','altitud','coordenada_utm_y','coordenada_utm_x','latitud','longitud']]
dfEstaciones=dfEstaciones.rename(columns={'codigo_estacion':'estacion','coordenada_utm_y':'utm_y','coordenada_utm_x':'utm_x',})
## Guardamos este dataframe en la tabla Estaciones de nuestra base de datos a través de la función insertaTabla.
cS.insertaTabla('Estaciones', dfEstaciones)


Cargamos el csv con los datos de los municipios que forman parte de la denominación de origen de la Pera de Rincón de Soto.


In [ ]:
## Definimos el directorio de trabajo
#os.chdir('D:\\Plagas')
dfMunicipios=pd.read_csv('Municipios.csv',sep=';',encoding='UTF-8')
## Guardamos este dataframe en la tabla Municipios de nuestra base de datos a través de la función insertaTabla.
cS.insertaTabla('Municipios', dfMunicipios)
## Faltaría asignar a cada municipio su estación de referencia. Lo haremos con SQL.

Calculamos las distancias y diferencias de altitud que hay de cada municipio a las estaciones meteorógicas del SIAR. Esta información es util para que los técnicos de la DOP puedan elegir qué estación es la más adecuada a cada municipio. Para ello definimos la función haversine en util que nos devuelve la distancia entre dos coordenada y la llamamos desde aquí.

In [ ]:
dfMunicipios=cS.getMunicipios()
dfEstaciones=cS.getEstaciones()
dfMunicipios=dfMunicipios.drop('Municipio',axis=1)
dfEstaciones=dfEstaciones.drop(['municipio','nombre','utm_y','utm_x'],axis=1)
## Hago el producto cartesiano de todas las estaciones y municipios
dfMunicipios['clave']=0
dfEstaciones['clave']=0
dfdistancia=pd.merge(dfMunicipios, dfEstaciones, on='clave').drop('clave',axis=1)
## Calculo las distancias y las diferencias de altitud
dfdistancia['distancia']=  dfdistancia.apply(lambda row: ut.haversine(row['Latitud'], row['Longitud'], row['latitud'], row['longitud']),  axis=1)
dfdistancia['difAltitud']=  dfdistancia.apply(lambda row: row['Altitud'] - row['altitud'],  axis=1)
dfdistancia=dfdistancia[['idMunicipio','estacion','distancia','difAltitud']]
cS.insertaTabla('DistanciasEst',dfdistancia)


A continuación, desde la página web del SIAR:
https://www.larioja.org/agricultura/es/informacion-agroclimatica/red-estaciones-agroclimaticas-siar/consulta-personalizada
Nos bajamos todos los datos climáticos de las estaciones que afectan a la DOP de la Pera de Rincón de Soto. Guardamos un fichero CSV con el formato <codigo_estacion>.csv y cargamos todos los ficheros en base de datos.

In [ ]:
## Definimos el directorio de trabajo
#os.chdir('D:\\Plagas\\Temperaturas Car')
## Leemos las estaciones que tenemos en base de datos
dfEstaciones=cS.getEstaciones()
## Por cada estacion leemos el csv correspondiente y guardamos en TemperaturasDiarias las temperaturas
for est in dfEstaciones['estacion']:
    dfTemperatura=pd.read_csv(str(est)+'.csv',sep=';',encoding='UTF-8')
    ## Convertimos la fecha, que está en string a formato fecha de base de datos.
    dfTemperatura['fecha']=dfTemperatura['fecha'].apply(lambda x: ut.convertirString(x,'CSV'))
    cS.insertaTabla('TemperaturasDiarias',dfTemperatura)

A partir de este momento, para actualizar los datos de las estaciones, utilizo las funciones de obtenerDatos: 
actualizarTodasEstaciones llama a la función actualizarEstacion que, por cada estación, me recupera toda la información meteorológica que le falta hasta la víspera de hoy.

El siguiente paso es cargar los datos de todas las capturas que los técnicos de la DOP guardan en ficheros Excel. Convertimos los ficheros en csv y los cargamos en base de datos.

In [4]:
## Leer ficheros csv con datos de vuelos de la DOP Pera
#os.chdir('D:\\Plagas\\CurvasDOP')
for i in range(2005,2024):
    ## En 2020 no se recogieron datos por motivo de la pandemia
    if i != 2020:
        fichero=str(i)+".csv"
        dfPolilla=pd.read_csv(fichero,sep=';',encoding='UTF-8')
        dfPolilla.head()
        dfPolilla['Fecha']=dfPolilla['Fecha'].apply(lambda x: ut.convertirString(str(x),'CSV'))
        columnas=list(dfPolilla.columns)
        ## Pivotamos los dataframe de forma que tengamos una fila por día y finca
        dfPolilla= pd.melt(dfPolilla, id_vars=['Fecha'], value_vars=columnas[1:len(columnas)], var_name='termino', value_name='Valor')
        cS.insertaTabla('Vuelos',dfPolilla)